In [563]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot
from sklearn.model_selection import cross_val_score

In [564]:
lr_data = pd.read_csv('hotel_booking.csv')


In [565]:
lr_data=lr_data[~((lr_data.children==0)&(lr_data.adults==0)&(lr_data.babies==0))]

In [566]:
lr_data['agent']=lr_data['agent'].fillna(0)
lr_data['company']=lr_data['company'].fillna(0)

In [567]:
lr_data = lr_data.dropna()

In [568]:
lr_data['agent']=lr_data['agent'].apply(lambda x:'Yes' if x!=0 else 'No')
lr_data['company']=lr_data['company'].apply(lambda x:'Yes' if x!=0 else 'No')

In [569]:
lr_data.meal.unique()

array(['BB', 'FB', 'HB', 'SC', 'Undefined'], dtype=object)

In [570]:
lr_data['meal'] = lr_data['meal'].apply(lambda x: 'SC' if x=='Unedfined' else x)

In [571]:
lr_data = lr_data[~(lr_data['market_segment']=='Undefined')]

In [572]:
lr_data['reserved_assigned']=lr_data.reserved_room_type==lr_data.assigned_room_type

In [573]:
lr_data = lr_data[lr_data.adr>0]

In [574]:
lr_data = lr_data[~(lr_data.distribution_channel=='Undefined')]

In [575]:
lr_data=lr_data.drop(columns=['reservation_status','name', 'email','phone-number', 'credit_card'])

In [576]:

lr_data.country = lr_data.country.apply(lambda x: 'PRT' if x=='PRT' else 'International')

In [577]:
lr_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116927 entries, 2 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           116927 non-null  object 
 1   is_canceled                     116927 non-null  int64  
 2   lead_time                       116927 non-null  int64  
 3   arrival_date_year               116927 non-null  int64  
 4   arrival_date_month              116927 non-null  object 
 5   arrival_date_week_number        116927 non-null  int64  
 6   arrival_date_day_of_month       116927 non-null  int64  
 7   stays_in_weekend_nights         116927 non-null  int64  
 8   stays_in_week_nights            116927 non-null  int64  
 9   adults                          116927 non-null  int64  
 10  children                        116927 non-null  float64
 11  babies                          116927 non-null  int64  
 12  meal            

In [578]:
lr_data['reservation_status_date']=pd.to_datetime(lr_data['reservation_status_date'])

In [579]:
pd.DatetimeIndex(lr_data['reservation_status_date']).day

Int64Index([ 2,  2,  3,  3,  3,  3,  6, 22, 23,  5,
            ...
             4,  5,  5,  6,  6,  6,  7,  7,  7,  7],
           dtype='int64', name='reservation_status_date', length=116927)

In [580]:
lr_data['reservation date'] = pd.DatetimeIndex(lr_data['reservation_status_date']).day
lr_data['reservation month'] = pd.DatetimeIndex(lr_data['reservation_status_date']).month
lr_data['reservation year'] = pd.DatetimeIndex(lr_data['reservation_status_date']).year

In [581]:
lr_data=lr_data.drop(columns=['reservation_status_date'])

In [582]:
lr_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116927 entries, 2 to 119389
Data columns (total 34 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           116927 non-null  object 
 1   is_canceled                     116927 non-null  int64  
 2   lead_time                       116927 non-null  int64  
 3   arrival_date_year               116927 non-null  int64  
 4   arrival_date_month              116927 non-null  object 
 5   arrival_date_week_number        116927 non-null  int64  
 6   arrival_date_day_of_month       116927 non-null  int64  
 7   stays_in_weekend_nights         116927 non-null  int64  
 8   stays_in_week_nights            116927 non-null  int64  
 9   adults                          116927 non-null  int64  
 10  children                        116927 non-null  float64
 11  babies                          116927 non-null  int64  
 12  meal            

In [583]:
### True False to 0 and 1
lr_data['reserved_assigned']=lr_data['reserved_assigned'].apply(lambda x: 'Yes' if x==True else 'No' )

In [603]:
X.res

,hotel,lead_time,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reserved_assigned,reservation date,reservation month
2,Resort Hotel,7,July,27,1,0,1,1,0.0,0,...,No,No,0,Transient,75.00,0,0,No,2,7
3,Resort Hotel,13,July,27,1,0,1,1,0.0,0,...,Yes,No,0,Transient,75.00,0,0,Yes,2,7
4,Resort Hotel,14,July,27,1,0,2,2,0.0,0,...,Yes,No,0,Transient,98.00,0,1,Yes,3,7
5,Resort Hotel,14,July,27,1,0,2,2,0.0,0,...,Yes,No,0,Transient,98.00,0,1,Yes,3,7
6,Resort Hotel,0,July,27,1,0,2,2,0.0,0,...,No,No,0,Transient,107.00,0,0,Yes,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,23,August,35,30,2,5,2,0.0,0,...,Yes,No,0,Transient,96.14,0,0,Yes,6,9
119386,City Hotel,102,August,35,31,2,5,3,0.0,0,...,Yes,No,0,Transient,225.43,0,2,Yes,7,9
119387,City Hotel,34,August,35,31,2,5,2,0.0,0,...,Yes,No,0,Transient,157.71,0,4,Yes,7,9
119388,City Hotel,109,August,35,31,2,5,2,0.0,0,...,Yes,No,0,Transient,104.40,0,0,Yes,7,9


In [584]:
X = lr_data.drop(columns=["is_canceled","arrival_date_year","reservation year"])
y = lr_data.is_canceled

In [585]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold,GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier

In [586]:
from sklearn.pipeline import make_pipeline

In [587]:
from sklearn.compose import make_column_transformer

In [588]:
cat_cols

Index(['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment',
       'distribution_channel', 'reserved_room_type', 'assigned_room_type',
       'deposit_type', 'agent', 'company', 'customer_type',
       'reserved_assigned'],
      dtype='object')

In [589]:
ohe = OneHotEncoder()
ohe.fit(X[['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment',
       'distribution_channel', 'reserved_room_type', 'assigned_room_type',
       'deposit_type', 'agent', 'company', 'customer_type',
       'reserved_assigned']])

OneHotEncoder()

In [590]:
ohe.categories_

[array(['City Hotel', 'Resort Hotel'], dtype=object),
 array(['April', 'August', 'December', 'February', 'January', 'July',
        'June', 'March', 'May', 'November', 'October', 'September'],
       dtype=object),
 array(['BB', 'FB', 'HB', 'SC', 'Undefined'], dtype=object),
 array(['International', 'PRT'], dtype=object),
 array(['Aviation', 'Complementary', 'Corporate', 'Direct', 'Groups',
        'Offline TA/TO', 'Online TA'], dtype=object),
 array(['Corporate', 'Direct', 'GDS', 'TA/TO'], dtype=object),
 array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'L'], dtype=object),
 array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L'],
       dtype=object),
 array(['No Deposit', 'Non Refund', 'Refundable'], dtype=object),
 array(['No', 'Yes'], dtype=object),
 array(['No', 'Yes'], dtype=object),
 array(['Contract', 'Group', 'Transient', 'Transient-Party'], dtype=object),
 array(['No', 'Yes'], dtype=object)]

In [591]:
column_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_),['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment',
       'distribution_channel', 'reserved_room_type', 'assigned_room_type',
       'deposit_type', 'agent', 'company', 'customer_type',
       'reserved_assigned']),remainder='passthrough')

In [592]:
from sklearn.preprocessing import OneHotEncoder

In [593]:
pipe = make_pipeline(column_trans,XGBClassifier())

In [611]:
pipe.fit(X,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['City Hotel', 'Resort Hotel'], dtype=object),
                                                                            array(['April', 'August', 'December', 'February', 'January', 'July',
       'June', 'March', 'May', 'November', 'October', 'September'],
      dtype=object),
                                                                            array(['BB', 'FB', 'HB', 'SC', 'Undefi...
                               feature_types=None, gamma=0, gpu_id=-1,
                               grow_policy='depthwise', importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_threshold=64, max_cat_to_onehot=4,
 

In [612]:
y_pred = pipe.predict(X)

In [597]:
import pickle
model = open('xgb.pickle','wb')
pickle.dump(pipe,model)
model.close()

In [598]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116927 entries, 2 to 119389
Data columns (total 31 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           116927 non-null  object 
 1   lead_time                       116927 non-null  int64  
 2   arrival_date_month              116927 non-null  object 
 3   arrival_date_week_number        116927 non-null  int64  
 4   arrival_date_day_of_month       116927 non-null  int64  
 5   stays_in_weekend_nights         116927 non-null  int64  
 6   stays_in_week_nights            116927 non-null  int64  
 7   adults                          116927 non-null  int64  
 8   children                        116927 non-null  float64
 9   babies                          116927 non-null  int64  
 10  meal                            116927 non-null  object 
 11  country                         116927 non-null  object 
 12  market_segment  

In [625]:
%%writefile deployment_model.py
import streamlit as st
import pandas as pd
st.title('Hotel Cancellation Predictor 🏨')
import pickle 

# Step 1: load the pickled model
model = open('xgb.pickle','rb')
xgbmodel=pickle.load(model)
model.close()

# Step 2: create a UI for front end user
from PIL import Image
hotel = st.radio("What's your Prefered Hotel type",('City Hotel','Resort Hotel'))
lead_time=st.slider('Enter Lead Time',1,200)
arrival_date_month=st.radio('Enter Arrival date month',('April', 'August', 'December', 'February', 'January', 'July',
        'June', 'March', 'May', 'November', 'October', 'September'))
arrival_date_week_number=st.slider('Enter Arrival week',1,54)
arrival_date_day_of_month=st.slider('Enter Arrival Day of month',1,31)
stays_in_weekend_nights= st.slider('Enter Weekend stays',1,108)
stays_in_week_nights = st.slider('Enter Weekday stays',1,300)
adults = st.slider('Enter Number of adults',1,25)
children =  st.slider('Enter Number of children',1,20)
babies =  st.slider('Enter Number of babies',0,20)
meal =  st.radio('Select Meal Type',('BB', 'FB', 'HB', 'SC'))
country=  st.radio('Select Country of Origin ',('PRT', 'International'))
market_segment=  st.radio('Select Market Segment',('Direct', 'Corporate', 'Online TA', 'Offline TA/TO',
       'Complementary', 'Groups', 'Undefined', 'Aviation'))
distribution_channel=  st.radio('Select Distribution Channel',('Direct', 'Corporate', 'TA/TO','GDS'))
is_repeated_guest_radio= st.radio('Repeated Guest ?',('Yes','No'))
if is_repeated_guest_radio=='Yes':
    is_repeated_guest=1
else:
     is_repeated_guest=0
    
    
previous_cancellations=  st.slider('Enter number of previous cancellations',1,30)
previous_bookings_not_canceled=  st.slider('Enter Number of previous bookings',1,50)
reserved_room_type=  st.radio('Reserved Room type',('C', 'A', 'D', 'E', 'G', 'F', 'H', 'L', 'P', 'B'))
assigned_room_type=  st.radio('Assigned Room type',('C', 'A', 'D', 'E', 'G', 'F', 'I', 'B', 'H', 'P', 'L', 'K'))
booking_changes=  st.slider('Enter changes made',1,100)
deposit_type=  st.radio("Choose Deposit type",('No Deposit', 'Refundable', 'Non Refund'))
agent=  st.radio("Booked via Agent ?",('Yes','No'))
company=  st.radio("Booked via company ?",('Yes','No'))
days_in_waiting_list=  st.slider('Enter Days in waiting list',1,800)
customer_type=  st.radio("Select Customer type ",('Transient', 'Contract', 'Transient-Party', 'Group'))
adr=  st.slider('Enter Cost per night',1,100)
required_car_parking_spaces=  st.slider('Number of car parking required',1,20)
total_of_special_requests=  st.slider('Number of special requests',1,50)
reserved_assigned=  st.radio("Is reserved room same as assigned room ?",('Yes','No'))
reservation_date=  st.slider('Reservation Date',1,31)
reservation_month=  st.slider('Reservation month',1,12)

# Step 3: Change user input as models input data
data={
'hotel' : hotel,
'lead_time': lead_time,
'arrival_date_month': arrival_date_month,
'arrival_date_week_number': arrival_date_week_number,
'arrival_date_day_of_month': arrival_date_day_of_month,
'stays_in_weekend_nights': stays_in_weekend_nights,
'stays_in_week_nights' : stays_in_week_nights,
'adults' : adults,
'children' : children,
'babies': babies,
'meal': meal,
'distribution_channel':distribution_channel,
'country': country,
'market_segment': market_segment ,
'is_repeated_guest' : is_repeated_guest ,                   
'previous_cancellations': previous_cancellations,
'previous_bookings_not_canceled':  previous_bookings_not_canceled,
'reserved_room_type': reserved_room_type,
'assigned_room_type': assigned_room_type,
'booking_changes':  booking_changes,
'deposit_type':  deposit_type,
'agent': agent,
'company': company ,
'days_in_waiting_list':  days_in_waiting_list,
'customer_type': customer_type,
'adr': adr,
'required_car_parking_spaces': required_car_parking_spaces,
'total_of_special_requests': total_of_special_requests,
'reserved_assigned':  reserved_assigned,
'reservation date': reservation_date,
'reservation month': reservation_month  
}

input_data = pd.DataFrame([data])
predictions = xgbmodel.predict(input_data)
if st.button('Predict'):
    if predictions==0:
        st.success('This Person is not likely to cancel')
    if predictions==1:
        st.error('High Chances of customer cancellation')

Overwriting deployment_model.py
